In [ ]:
import requests 
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

In [ ]:
url='https://www.flipkart.com/search?q=new+launch+tv&sid=ckf%2Cczl&as=on&as-show=on&otracker=AS_QueryStore_OrganicAutoSuggest_1_12_na_na_ps&otracker1=AS_QueryStore_OrganicAutoSuggest_1_12_na_na_ps&as-pos=1&as-type=RECENT&suggestionId=new+launch+tv%7CTelevisions&requestId=33ae24d5-65fa-4e7d-ae1b-36f74371f62d&as-backfill=on&otracker=nmenu_sub_TVs%20%26%20Appliances_0_New%20Launches'
r=requests.get(url).text
# print(r)

## 1. Flipcart Scaping 
#### A) Searching through TAGS 

In [ ]:
# url = 'https://www.flipkart.com/search?q=new+launch+tv&sid=ckf%2Cczl&as=on&as-show=on&otracker=AS_QueryStore_OrganicAutoSuggest_1_12_na_na_ps&otracker1=AS_QueryStore_OrganicAutoSuggest_1_12_na_na_ps&as-pos=1&as-type=RECENT&suggestionId=new+launch+tv%7CTelevisions&requestId=33ae24d5-65fa-4e7d-ae1b-36f74371f62d&as-backfill=on&otracker=nmenu_sub_TVs%20%26%20Appliances_0_New%20Launches'
url="https://www.flipkart.com/search?q=new+launch+tv&sid=ckf%2Cczl&as=on&as-show=on&otracker=AS_QueryStore_OrganicAutoSuggest_1_12_na_na_ps&otracker1=AS_QueryStore_OrganicAutoSuggest_1_12_na_na_ps&as-pos=1&as-type=RECENT&suggestionId=new+launch+tv%7CTelevisions&requestId=33ae24d5-65fa-4e7d-ae1b-36f74371f62d&as-backfill=on&otracker=nmenu_sub_TVs%20%26%20Appliances_0_New%20Launches"
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
}
r = requests.get(url, headers=headers)

In [ ]:
soup=BeautifulSoup(r.text, 'lxml')
Product_name=soup.find_all('div', {'class':'KzDlHZ'})
product_list=[]
for product in Product_name:
    na=product.string
    # print(na)
    product_list.append(na)  # or use price.text

prices=soup.find_all('div', {'class':'Nx9bqj _4b5DiR'})
price_list=[]
for price in prices:
    pr=price.string
    price_list.append(pr)  # or use price.text

df=pd.DataFrame({'Product Name': product_list, 'Price':price_list})
# df.to_csv("C:\\Users\\Rajesh Umrao\\OneDrive\\Desktop\\Product.csv", index=False, encoding='utf-8')
df

#### B) Searching Through Regression expression 

In [ ]:
# pattern = re.compile(r'Price')
# divs = soup.find_all('div', text=pattern)

#### C) Searching in case of Nested HTML Tags 

In [ ]:
from bs4 import BeautifulSoup

html = """
<div class="product">
    <div class="name">Product Name</div>
    <div class="price">₹8,999</div>
    <div class="details">
        <p>Detail 1</p>
        <p>Detail 2</p>
    </div>
</div>
"""

soup = BeautifulSoup(html, 'lxml')

# Access the parent product div
product_div = soup.find('div', class_='product')

# Navigate to child elements
name_div = product_div.find('div', class_='name')
price_div = product_div.find('div', class_='price')
details_div = product_div.find('div', class_='details')

print("Product Name:", name_div.text)
print("Price:", price_div.text)

# Navigate to nested details
details = [p.text for p in details_div.find_all('p')]
print("Details:", details)


### Extracting the Data from Table of the website 

In [ ]:
url="https://www.iplt20.com/auction/2024"
r=requests.get(url)
# print(r)

soup=BeautifulSoup(r.text,"lxml")
table=soup.find('table', class_='ih-td-tab auction-tbl')

#Extract the headers of the table 
cols=[]
for i in table.find_all('th'):
    col=i.text
    cols.append(col)
# print(cols)

# Extract rows
rows = []
for tr in table.find_all('tr'):
    cells = tr.find_all('td')
    if len(cells) > 0:
        row = [cell.text.strip() for cell in cells]
        rows.append(row)
# print(rows)
#convert headers list
df=pd.DataFrame(rows, columns=cols)
# df.to_csv("C:\\Users\\Rajesh Umrao\\OneDrive\\Desktop\\IPL.csv", index=False, encoding='utf-8')



### Extracting the Data from multiple Pages 

In [ ]:
url='https://www.flipkart.com/search?q=mobile&otracker=search&otracker1=search&marketplace=FLIPKART&as-show=on&as=off&sort=popularity'
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"}
r = requests.get(url, headers=headers)
soup=BeautifulSoup(r.text, 'lxml')

link = soup.find("div", class_="DOjaWF gdgoEp").find("a", class_="_9QVEpD").get("href")
base_url="https://www.flipkart.com"+link

#making general base link
link=base_url.strip("2")
#create empty list for using in forloop condition
product_list=[]
price_list=[]

for i in range(1,2):
    pg_link= link + str(i)
    rq = requests.get(pg_link, headers=headers)
    final_soup=BeautifulSoup(rq.text, 'lxml')
    Product_name=final_soup.find_all('div', {'class':'KzDlHZ'})
    for product in Product_name:
        name=product.string
        product_list.append(name)  

    prices=soup.find_all('div', {'class':'Nx9bqj _4b5DiR'})
    for price in prices:
        pr=price.string
        price_list.append(pr) 
    
print(len(product_list))
print(len(price_list))
df=pd.DataFrame({'Product Name': product_list, 'Price':price_list})
# df.to_csv("C:\\Users\\Rajesh Umrao\\OneDrive\\Desktop\\Product.csv", index=False, encoding='utf-8')


# Selenium 

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# Initialize the WebDriver
driver = webdriver.Chrome()

# Open the desired URL
driver.get("https://www.uppclonline.com")

# Wait for the button to be clickable
wait = WebDriverWait(driver,0)
button = wait.until(EC.element_to_be_clickable((By.XPATH, "/html/body/div[2]/div/div[3]/div[1]/div[4]/div/div[2]/div/div/div/form/div/a[1]/span")))

# Click the button
button.click()



In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
import time
from PIL import Image
import pytesseract
import requests

# Set the path to the Tesseract executable (adjust this for your system)
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'  # Windows path example

# Initialize the WebDriver
driver = webdriver.Chrome()

# Open the desired URL
driver.get("https://www.uppclonline.com")

# Wait for the button to be clickable and click it
wait = WebDriverWait(driver,10)
button = wait.until(EC.element_to_be_clickable((By.XPATH, "/html/body/div[2]/div/div[3]/div[1]/div[4]/div/div[2]/div/div/div/form/div/a[1]/span")))
button.click()

# Wait for the dropdown menu to be present
dropdown = wait.until(EC.presence_of_element_located((By.XPATH, "/html/body/div[2]/div/div[3]/div/div[3]/div[5]/div/div[2]/div/form/table/tbody/tr[3]/td[3]/div/select")))

# Create a Select object
select = Select(dropdown)
select.select_by_index(1)  # Index is 0-based, so index 1 selects the 2nd option

# Input the account number
account_number_field = wait.until(EC.presence_of_element_located((By.XPATH, "/html/body/div[2]/div/div[3]/div/div[3]/div[5]/div/div[2]/div/form/table/tbody/tr[5]/td[3]/div/input")))
account_number_field.send_keys("6048778000")

# Input the password
password_field = wait.until(EC.presence_of_element_located((By.XPATH, "/html/body/div[2]/div/div[3]/div/div[3]/div[5]/div/div[2]/div/form/table/tbody/tr[7]/td[3]/div/input")))
password_field.send_keys("anuj5395*")



In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
import time
from PIL import Image
import pytesseract
import requests

# Set the path to the Tesseract executable (adjust this for your system)
pytesseract.pytesseract.tesseract_cmd = r"C:/Users/Rajesh Umrao/Downloads/tesseract-ocr-setup-3.02.02.exe"  # Windows path example

# Initialize the WebDriver
driver = webdriver.Chrome()

# Open the desired URL
driver.get("https://www.uppclonline.com")

# Wait for the button to be clickable and click it
wait = WebDriverWait(driver,10)
button = wait.until(EC.element_to_be_clickable((By.XPATH, "/html/body/div[2]/div/div[3]/div[1]/div[4]/div/div[2]/div/div/div/form/div/a[1]/span")))
button.click()

# Wait for the dropdown menu to be present
dropdown = wait.until(EC.presence_of_element_located((By.XPATH, "/html/body/div[2]/div/div[3]/div/div[3]/div[5]/div/div[2]/div/form/table/tbody/tr[3]/td[3]/div/select")))

# Create a Select object
select = Select(dropdown)
select.select_by_index(1)  # Index is 0-based, so index 1 selects the 2nd option

# Input the account number
account_number_field = wait.until(EC.presence_of_element_located((By.XPATH, "/html/body/div[2]/div/div[3]/div/div[3]/div[5]/div/div[2]/div/form/table/tbody/tr[5]/td[3]/div/input")))
account_number_field.send_keys("7146840100")

# Input the password
password_field = wait.until(EC.presence_of_element_located((By.XPATH, "/html/body/div[2]/div/div[3]/div/div[3]/div[5]/div/div[2]/div/form/table/tbody/tr[7]/td[3]/div/input")))
password_field.send_keys("anuj5395*")

